# Useful Pandas Snippets

- From [Pandas Snippets](https://github.com/siebenrock/pandas-snippets) by [Kai Siebenrock](https://github.com/siebenrock)

Updated by [John Fogarty](https://github.com/jfogarty) for Python 3.6 and [Base2 MLI](https://github.com/base2solutions/mli) and [colab](https://colab.research.google.com) standalone evaluation.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Importing Data

In [22]:
import os
REPODATA='https://github.com/plotly/datasets/blob/master/titanic.csv'
RAWDATA='https://raw.githubusercontent.com/plotly/datasets/master/titanic.csv'
filename='titanic.csv'
TMPDATA='./tmpData'
if not os.path.exists(TMPDATA) : os.makedirs(TMPDATA)
datafile=os.path.join(TMPDATA, filename)
!curl $RAWDATA -o $datafile

In [23]:
#@title Nasty File Transfer Utility Tools
import numpy as np
import requests
import shutil
import gzip
import os
from bs4 import BeautifulSoup

ds = np.DataSource()
def copyHere(URL, toPath, quiet=False):
    toDir, toFile = os.path.split(toPath)
    toPath = os.path.join(toDir, toFile)
    if os.path.exists(toPath + 'x'):
        if not quiet:
            print(f"- Skipped copy of existing file {toPath}.")
    else:
        if ds.exists(URL):
            if not toFile:
                urlPrefix, toFile = os.path.split(URL)
            response = requests.get(URL, stream=True)
            response.raw.decode_content=True  # reflate and ungzip automatically.
            response.encoding = 'utf-8'
            if toDir:
                if not os.path.exists(toDir): 
                  print(f"- Creating directory '{toDir}'.")
                  os.makedirs(toDir)
            with open(toPath, 'wb') as f: shutil.copyfileobj(response.raw, f)
            if not quiet: 
                print(f"- Downloaded {URL}.")
            gzipped = False
            # If the file sent is gzipped, unpack it anyway.
            with open(toPath, 'rb') as fin:
                prefix = fin.read(2) 
                gzipped = prefix == b'\x1f\x8b'
            if gzipped:
                gzPath = toPath + '.gz'               
                if os.path.exists(gzPath):
                    os.remove(gzPath)
                os.rename(toPath, gzPath)
                with gzip.open(gzPath) as gz:
                    with open(toPath, 'wb') as fout:
                         shutil.copyfileobj(gz, fout)
                if not quiet: 
                    print(f"- Unpacked gzipped file '{gzPath}' to '{toPath}'.")
            else:
                print(f"- Installed locally as '{toPath}'.")
        else:
            print(f"** Sorry, can't copy '{URL}' to '{toPath}'.")

In [24]:
copyHere(RAWDATA, datafile)

- Downloaded https://raw.githubusercontent.com/plotly/datasets/master/titanic.csv.
- Installed locally as './tmpData\titanic.csv'.


Read from CSV file

In [25]:
df = pd.read_csv(datafile)



In [26]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Creating Data

Using dataframe

In [28]:
pd.DataFrame({'Name':['Marie', 'John', 'Max', 'Jane'],
              'Age':[32, 28, 27, 33]}, 
             index=['rank1','rank2','rank3','rank4'])

,Name,Age
rank1,Marie,32
rank2,John,28
rank3,Max,27
rank4,Jane,33


In [29]:
pd.DataFrame(np.random.randint(low=0, high=100, size=(5, 5)), 
             columns=['A', 'B', 'C', 'D', 'E'])

,A,B,C,D,E
0,1,0,99,75,79
1,79,90,36,94,7
2,9,20,42,20,13
3,3,70,27,76,38
4,24,23,19,88,84


Using list comprehension

In [30]:
list = [x**2 for x in range(10)]

In [31]:
[x for x in list if x % 2 == 0]

[0, 4, 16, 36, 64]

## Cleaning

Drop NaN in fare

In [32]:
df.dropna(subset=["Fare"], inplace=True)

Return null values

In [33]:
df[df['Fare'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Upper case all column names

In [34]:
df.columns = map(str.upper, df.columns)
df.head()

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Rename columns

In [35]:
df = df.rename(columns = {
    'Pclass':'Class',
    'Name':'Full Name',
})

Alternatively

In [36]:
df.columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
df.columns = ['Id', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 
              'Parents/Children Aboard', 'Ticket', 'Fare', 'Cabin', 'Embarked']

Filter columns containing "Aboard"

In [37]:
df_aboard = df.loc[:, df.columns[df.columns.str.contains('Aboard')].tolist()]

In [38]:
df_aboard.head()

,Siblings/Spouses Aboard,Parents/Children Aboard
0,1,0
1,1,0
2,0,0
3,1,0
4,0,0


Replace strings in column

In [39]:
df['Sex'] = df['Sex'].str.replace('Mr.', 'Mister')

Remove if contains character

In [40]:
files = ['afile', 'bfile', 'not~mefile', 'cfile']
notfiles = [file for file in files if "~" not in file]
notfiles

['afile', 'bfile', 'cfile']

Remove based on multiple values

In [41]:
df = df[~df['Name'].isin(['Invalid', 'Unknown'])]
df

,Id,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Change type

In [42]:
df['Fare'] = df['Fare'].astype(float)

Reset index

In [43]:
df.reset_index(drop=True, inplace=True)

Convert to lower case

In [44]:
df['Sex'] = df['Sex'].str.lower()

In [45]:
df["Pclass"] * 1000

0      3000
1      1000
2      3000
3      1000
4      3000
       ... 
151    1000
152    3000
153    3000
154    3000
155    1000
Name: Pclass, Length: 156, dtype: int64

In [46]:
df.head()

,Id,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Deleting columns

In [47]:
del df['Siblings/Spouses Aboard']
del df['Parents/Children Aboard']

## Exploring

Number of rows

In [48]:
len(df.index)

156

Get info

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 10 columns):
Id          156 non-null int64
Survived    156 non-null int64
Pclass      156 non-null int64
Name        156 non-null object
Sex         156 non-null object
Age         126 non-null float64
Ticket      156 non-null object
Fare        156 non-null float64
Cabin       31 non-null object
Embarked    155 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 12.3+ KB


Describe data

In [50]:
df.describe()

,Id,Survived,Pclass,Age,Fare
count,156.000000,156.000000,156.000000,126.000000,156.000000
mean,78.500000,0.346154,2.423077,28.141508,28.109587
std,45.177428,0.477275,0.795459,14.613880,39.401047
min,1.000000,0.000000,1.000000,0.830000,6.750000
25%,39.750000,0.000000,2.000000,19.000000,8.003150
50%,78.500000,0.000000,3.000000,26.000000,14.454200
75%,117.250000,1.000000,3.000000,35.000000,30.371850
max,156.000000,1.000000,3.000000,71.000000,263.000000


Select two columns

In [51]:
df[['Name', 'Fare']].head()

,Name,Fare
0,"Braund, Mr. Owen Harris",7.2500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",71.2833
2,"Heikkinen, Miss. Laina",7.9250
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",53.1000
4,"Allen, Mr. William Henry",8.0500


Get titles

In [52]:
df["Title"] = df["Name"].str.split(" ").str[0]

Looking only at males

In [53]:
df[df['Sex'] == 'male'].head()

,Id,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,"Braund,"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,"Allen,"
5,6,0,3,"Moran, Mr. James",male,NaN,330877,8.4583,NaN,Q,"Moran,"
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,17463,51.8625,E46,S,"McCarthy,"
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,349909,21.0750,NaN,S,"Palsson,"


Looking only at males who survived

In [54]:
df[(df['Sex'] == 'male') & (df['Survived'] == 1)].head()

,Id,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Title
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,244373,13.0000,NaN,S,"Williams,"
21,22,1,2,"Beesley, Mr. Lawrence",male,34.0,248698,13.0000,D56,S,"Beesley,"
23,24,1,1,"Sloper, Mr. William Thompson",male,28.0,113788,35.5000,A6,S,"Sloper,"
36,37,1,3,"Mamee, Mr. Hanna",male,NaN,2677,7.2292,NaN,C,"Mamee,"
55,56,1,1,"Woolner, Mr. Hugh",male,NaN,19947,35.5000,C52,S,"Woolner,"


Looking only at males who survived above the age of 50

In [55]:
df[(df['Sex'] == 'male') & (df['Survived'] == 1) & (df['Age'] > 50)].head()

,Id,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Title


Set column value based on other columns

In [56]:
df['Note'] = np.nan

In [57]:
df.loc[(df['Sex'] == 'male') & (df['Survived'] == 1) & (df['Age'] > 50), 
       ['Note']] = 'Male Above 50 Survived'

In [58]:
df['Note'].sort_values()[:3]

0    NaN
1    NaN
2    NaN
Name: Note, dtype: object

Number of men who survived

In [59]:
len(df[(df['Sex'] == 'male') & (df['Survived'] == 1)])

14

Average age of men who survived

In [60]:
df[(df['Sex'] == 'male') & (df['Survived'] == 1)]['Age'].mean()

23.314444444444444

Filter by multiple values

In [61]:
df[df["Name"].isin(["Mr. Charles Eugene Williams", "Mr. Lawrence Beesley"])]

,Id,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,Note


Highest fare paid

In [62]:
df.loc[df['Fare'].idxmax()]

Id                                      28
Survived                                 0
Pclass                                   1
Name        Fortune, Mr. Charles Alexander
Sex                                   male
Age                                     19
Ticket                               19950
Fare                                   263
Cabin                          C23 C25 C27
Embarked                                 S
Title                             Fortune,
Note                                   NaN
Name: 27, dtype: object

Sorting

In [63]:
df.sort_values(['Fare', 'Age'], ascending=[0,1]).head()

,Id,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,Note
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,19950,263.0000,C23 C25 C27,S,"Fortune,",NaN
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,19950,263.0000,C23 C25 C27,S,"Fortune,",NaN
118,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,PC 17558,247.5208,B58 B60,C,"Baxter,",NaN
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,PC 17569,146.5208,B78,C,"Spencer,",NaN
62,63,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,36973,83.4750,C83,S,"Harris,",NaN


Sort by multiple columns

In [64]:
df.sort_values(['Fare', 'Age'], ascending=[0,1]).head()

,Id,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,Note
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,19950,263.0000,C23 C25 C27,S,"Fortune,",NaN
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,19950,263.0000,C23 C25 C27,S,"Fortune,",NaN
118,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,PC 17558,247.5208,B58 B60,C,"Baxter,",NaN
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,PC 17569,146.5208,B78,C,"Spencer,",NaN
62,63,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,36973,83.4750,C83,S,"Harris,",NaN


Number of classes

In [65]:
df['Pclass'].unique()

array([3, 1, 2], dtype=int64)

Count of each class

In [66]:
df['Pclass'].value_counts()

3    96
2    30
1    30
Name: Pclass, dtype: int64

Find duplicates : [pandas.DataFrame.duplicated](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html)

In [67]:
df[df.duplicated(['Name'], keep=False)]

,Id,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,Note


## Looping

In [68]:
for index, row in df.iterrows():
    continue
#     print(index)
#     print(row)

Returning tuples

In [69]:
for row in df.itertuples():
    continue
#     print(row)

## Grouping

Group by class and aggregate fare by mean

In [70]:
df.groupby(['Pclass'])['Fare'].mean()

Pclass
1    79.50194
2    23.65972
3    13.44006
Name: Fare, dtype: float64

Pivot table

In [71]:
pd.pivot_table(df, values='Fare', index='Pclass', columns='Sex', 
               aggfunc=np.mean)

Sex,female,male
Pclass,,
1,90.007400,74.999600
2,21.325000,25.216200
3,15.080834,12.498633


Sample weighted average aggregation function

In [72]:
agg_func = {'colA': ['sum'], 
            'colB': lambda x: np.average(x, weights=d.loc[x.index, 'colC'])}

## Miscellaneous

Functions in dictionary

In [73]:
func = {
    'times2': lambda x: print("The solution is: {}".format(x**2)),
    'times3': lambda x: print("The solution is: {}".format(x**3)),
    'times4': lambda x: print("The solution is: {}".format(x**4))
}

In [74]:
func['times2'](3)

The solution is: 9


## Recommended Cheat Sheets

* [Pandas DataFrame Object](http://www.webpages.uidaho.edu/~stevel/504/Pandas%20DataFrame%20Notes.pdf) from University of Idaho
* [Data Wrangling with Pandas](http://cs.umw.edu/~stephen/cpsc219/Pandas_Cheat_Sheet.pdf) from University of Mary Washington
* [Python for Data Science Pandas Basics](http://datacamp-community.s3.amazonaws.com/3857975e-e12f-406a-b3e8-7d627217e952) from DataCamp
* [Data Science Python Intermediate](https://www.dataquest.io/blog/large_files/python-cheat-sheet-intermediate.pdf) from Dataquest
* [Data Science Numpy](https://www.dataquest.io/blog/large_files/numpy-cheat-sheet.pdf) from Dataquest
* [Data Science Pandas](https://www.dataquest.io/blog/large_files/pandas-cheat-sheet.pdf) from Dataquest

### End of notebook.